<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%2010/cis530_nmt_mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/Drive', force_remount = True)
# When you run this block, you will need to click open a link to get some 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [0]:
import os
# TODO
os.chdir('Drive/My Drive/CIS-530/Homework 10/data')

In [3]:
!pip install mecab-python3

     |████████████████████████████████| 17.1MB 202kB/s 


In [0]:
import MeCab

wakati = MeCab.Tagger("-Owakati")

for fname in ['ja-train.txt','ja-dev.txt','ja-test.txt']:
  with open('split/' + fname, encoding = 'utf=8') as fj_orig:
    with open('mecab/mecab-' + fname, 'w', encoding = 'utf-8') as fj_tok: 
      for line in fj_orig:
        tokens = wakati.parse(line)
        fj_tok.write(tokens)

In [5]:
# Install BPE (https://github.com/rsennrich/subword-nmt)
# TODO
!pip install subword-nmt

In [6]:
# Install OpenNMT (https://github.com/OpenNMT/OpenNMT-py)
# TODO
!pip install OpenNMT-py

     |████████████████████████████████| 194kB 9.4MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 2.2MB 12.0MB/s 
     |████████████████████████████████| 153kB 45.9MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.2.1-cp36-none-any.whl size=19266 sha256=75aa1bb7ee9bd7df86c8ee0fbc90aed706ec68caf1289b7f6028768abe94131e
  Stored in directory: /root/.cache/pip/wheels/76/32/15/c284be210ac2e0b87d486c59eda286976b76275cd86f81b59d
Successfully built configargparse
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.30.0 which is incompatible.
  Found existing installation: tqdm 4.38.0
    Uninstalling tqdm-4.38.0:
      Successfully uninstalled tqdm-4.38.0
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
# Apply BPE, following instructions from the Github repo
# this takes around 15 minutes

# TODO
# codes_file is where the BPE information is stored. Name it as you like. 

# Learn and apply BPE for English data

!subword-nmt learn-bpe -s 10000 < 'mecab/mecab-ja-train.txt' > 'mecab/mecab-ja-bpe' # learn from train
!subword-nmt apply-bpe -c 'mecab/mecab-ja-bpe' < 'mecab/mecab-ja-train.txt' > 'mecab/mecab-ja-train-bpe-out.txt' # apply to train
!subword-nmt apply-bpe -c 'mecab/mecab-ja-bpe' < 'mecab/mecab-ja-dev.txt' > 'mecab/mecab-ja-dev-bpe-out.txt' # apply to dev
!subword-nmt apply-bpe -c 'mecab/mecab-ja-bpe' < 'mecab/mecab-ja-test.txt' > 'mecab/mecab-ja-test-bpe-out.txt' # apply to test

In [9]:
# Run OpenNMT, following instructions from the Github repo
# this takes 1-2 hours

# TODO
# Name the prefixes for data and checkpoints as you like
!onmt_preprocess -train_src 'mecab/mecab-ja-train-bpe-out.txt' -train_tgt 'bpe/en-train-bpe-out.txt' -valid_src 'mecab/mecab-ja-dev-bpe-out.txt' -valid_tgt 'bpe/en-dev-bpe-out.txt' -save_data 'mecab/demo' -overwrite

[2020-04-16 01:47:16,441 INFO] Extracting features...
[2020-04-16 01:47:16,459 INFO]  * number of source features: 0.
[2020-04-16 01:47:16,459 INFO]  * number of target features: 0.
[2020-04-16 01:47:16,459 INFO] Building `Fields` object...
[2020-04-16 01:47:16,459 INFO] Building & saving training data...
[2020-04-16 01:47:18,233 INFO] Building shard 0.
[2020-04-16 01:47:52,639 INFO]  * saving 0th train data shard to alternative_1/demo.train.0.pt.
tcmalloc: large alloc 1073741824 bytes == 0x66cc2000 @  0x7fd45f7771e7 0x5ad4cb 0x4e0bf6 0x4e291a 0x4e34a4 0x4e307c 0x4e14d8 0x4e2c5b 0x4e2aab 0x4e30d0 0x4e14d8 0x4e2c5b 0x4e3ac6 0x5ebdc2 0x50a94c 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x508245 0x589471 0x5a067e
[2020-04-16 01:48:18,413 INFO] Building shard 1.
[2020-04-16 01:48:51,656 INFO]  * saving 1th train data shard to alternative_1/demo.train.1.pt.
[2020-04-16 01:49:16,392 INFO] Building shard 2.
[2020-04-16 0

In [10]:
!onmt_train -gpu_ranks 0 -data 'mecab/demo' -save_model demo-model

[2020-04-16 01:50:25,306 INFO]  * src vocab size = 19751
[2020-04-16 01:50:25,308 INFO]  * tgt vocab size = 10099
[2020-04-16 01:50:25,309 INFO] Building model...
[2020-04-16 01:50:34,689 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(19751, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(10099, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
     

In [18]:
!onmt_translate -model 'mecab/demo-model_step_100000.pt' -src 'mecab/mecab-ja-test.txt' -output 'mecab/pred-mecab.txt' -replace_unk -verbose

Streaming output truncated to the last 5000 lines.
PRED 751: sweet boy.
PRED SCORE: -3.7648

SENT 752: ['ショー', 'ン', 'が', 'これ', 'を', 'くれ', 'た', 'ん', 'だ']
PRED 752: sha@@ n gave me this.
PRED SCORE: -4.7683

SENT 753: ['そっち', 'へ', '向かう', '途中', 'よ', 'ベック']
PRED 753: we're on the way.
PRED SCORE: -7.2062

SENT 754: ['そう', 'だ', 'ベン', '送電', '線', 'に', '従え']
PRED 754: that's right.
PRED SCORE: -5.2197

SENT 755: ['情け', 'を', 'かけ', 'て', 'くれる', 'よう', 'に', 'と', '猫', 'ちゃんと', 'ドン', 'キー', 'に', '祈っ', 'て', 'くれ', '!']
PRED 755: please pra@@ ise the cat and pray for the cat@@ !
PRED SCORE: -14.0409

SENT 756: ['「', 'だから', 'ね', '」', 'その', '子', 'に', '言っ', 'て', 'あげる', '「', 'ママ', 'は', '心配', '性', 'だ', 'けど']
PRED 756: that's why i told her that she was worried
PRED SCORE: -11.2027

SENT 757: ['車', 'の', 'コマーシャル', 'の', '?']
PRED 757: is that the car of the car?
PRED SCORE: -10.0852

SENT 758: ['勉強', 'し', '始め', 'まし', 'た']
PRED 758: i started studying
PRED SCORE: -4.9678

SENT 759: ['だが', '俺', 'に', 'は', 'できる', 'ん'

In [0]:
# reverse BPE
!sed -r 's/(@@ )|(@@ ?$)//g' 'mecab/pred-mecab.txt' > 'mecab/pred.txt'

In [20]:
# Evaluate translations using BLEU
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/generic/multi-bleu.perl
# TODO
!perl multi-bleu.perl 'split/en-test.txt' < 'mecab/pred.txt'

--2020-04-16 02:48:25--  https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/generic/multi-bleu.perl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5234 (5.1K) [text/plain]
Saving to: ‘multi-bleu.perl.1’

multi-bleu.perl.1   100%[===================>]   5.11K  --.-KB/s    in 0.001s  

2020-04-16 02:48:25 (6.19 MB/s) - ‘multi-bleu.perl.1’ saved [5234/5234]

BLEU = 5.93, 29.5/11.9/5.5/2.3 (BP=0.730, ratio=0.761, hyp_len=10356, ref_len=13617)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl 

In [22]:
# Run NMT system on jp_test_sentences.txt 

wakati = MeCab.Tagger("-Owakati")

with open('jp_test_sentences.txt', encoding = 'utf-8') as test_file:
  with open('jp-test-sentences-mecab.txt', 'w', encoding = 'utf-8') as mecab_test_file:
    for line in test_file:
      tokens = wakati.parse(line)
      mecab_test_file.write(tokens)

# BPE for jp-test-sentences-mecab.txt
!subword-nmt apply-bpe -c 'mecab/mecab-ja_bpe' < 'jp-test-sentences-mecab.txt' > 'jp-test-sentences-mecab-bpe-out.txt' # apply to test
!onmt_translate -model 'mecab/demo-model_step_100000.pt' -src 'jp-test-sentences-mecab-bpe-out.txt' -output 'translations-mecab-bpe.txt' -replace_unk -verbose
!sed -r 's/(@@ )|(@@ ?$)//g' 'translations-mecab-bpe.txt' > 'translations.txt'

Streaming output truncated to the last 5000 lines.
PRED 1251: the cat sleep under bed.
PRED SCORE: -3.8371

SENT 1252: ['私', 'の', '白い', 'シャツ', 'は', 'どこ', 'です', 'か', '？']
PRED 1252: where's my white shir@@ t?
PRED SCORE: -2.2767

SENT 1253: ['彼ら', 'は', 'リンゴ', 'を', '取っ', 'た', '。']
PRED 1253: they took the ap@@ ple.
PRED SCORE: -2.9891

SENT 1254: ['私', 'は', '土地', '線', 'を', '持っ', 'て', 'い', 'ます', '。']
PRED 1254: i have a land line.
PRED SCORE: -2.7581

SENT 1255: ['彼女', 'は', '彼', 'の', '家族', 'と', '一緒', 'に', '住ん', 'で', 'い', 'ます', '。']
PRED 1255: she lives with his family.
PRED SCORE: -1.0799

SENT 1256: ['彼', 'は', '3', '人', 'の', '兄弟', 'を', '持っ', 'て', 'い', 'ます', '。']
PRED 1256: he has three brother@@ s.
PRED SCORE: -2.2399

SENT 1257: ['歴史', '的', 'な', '証拠', 'は', 'あり', 'ませ', 'ん', '。']
PRED 1257: there's no history of history.
PRED SCORE: -4.4439

SENT 1258: ['男', 'は', '軍事', 'コート', 'を', '持っ', 'て', 'い', 'ます', '。']
PRED 1258: a man has a military co@@ at.
PRED SCORE: -3.2401

SENT 1259: ['それ', 'は